In [1]:
import difflib

In [ ]:


def find_similar_merchants_simple(merchant_list, threshold=0.8):
    """
    Groups similar merchant names based on a simple pairwise similarity ratio.
    
    Args:
        merchant_list (list): The list of merchant names (strings).
        threshold (float): The minimum similarity ratio (0.0 to 1.0) to group names.
    
    Returns:
        tuple: (list of similar groups, list of unique names)
    """
    # Create a list of indices to keep track of which names have been grouped
    remaining_indices = list(range(len(merchant_list)))
    
    # Store the groups of similar names
    similar_groups = []
    
    # Process the list until all names are either grouped or deemed unique
    while remaining_indices:
        # Start a new group with the first remaining name
        current_index = remaining_indices.pop(0)
        current_name = merchant_list[current_index]
        
        # This will be the first group member
        current_group = {current_name}
        
        # Iterate through the rest of the remaining names to find matches
        i = 0
        while i < len(remaining_indices):
            check_index = remaining_indices[i]
            check_name = merchant_list[check_index]
            
            # Calculate the similarity ratio
            ratio = difflib.SequenceMatcher(None, current_name.lower(), check_name.lower()).ratio()
            
            if ratio >= threshold:
                # Add to the group and remove from the remaining list
                current_group.add(check_name)
                remaining_indices.pop(i) # Use pop(i) to remove the element
            else:
                # Only increment if we didn't remove an element
                i += 1
        
        # Add the group to the results
        similar_groups.append(list(current_group))

    # Identify which groups are truly "similar" (size > 1) and which are "unique" (size = 1)
    similar_merchants = [group for group in similar_groups if len(group) > 1]
    unique_merchants = [group[0] for group in similar_groups if len(group) == 1]
    
    return similar_merchants, unique_merchants

# --- Example Usage ---
merchants = [
    "Starbucks Coffee",
    "StarBucks",
    "The Home Depot",
    "HomeDepot",
    "Lowe's",
    "Best Buy Co.",
    "Best Buy"
]

similar, unique = find_similar_merchants_simple(merchants, threshold=0.85)

print("## Simple Pairwise Comparison (Threshold 0.55) ☕️")
print(f"Similar Merchants:\n{similar}")
print(f"Unique Merchants:\n{unique}")

## Simple Pairwise Comparison (Threshold 0.85) ☕️
Similar Merchants:
[]
Unique Merchants:
['Starbucks Coffee', 'StarBucks', 'The Home Depot', 'HomeDepot', "Lowe's", 'Best Buy Co.', 'Best Buy']


In [3]:
similar, unique = find_similar_merchants_simple(merchants, threshold=0.55)
print("## Simple Pairwise Comparison (Threshold 0.55) ☕️")
print(f"Similar Merchants:\n{similar}")
print(f"Unique Merchants:\n{unique}")

## Simple Pairwise Comparison (Threshold 0.55) ☕️
Similar Merchants:
[['Starbucks Coffee', 'StarBucks'], ['The Home Depot', 'HomeDepot'], ['Best Buy Co.', 'Best Buy']]
Unique Merchants:
["Lowe's"]


In [13]:
merchants = [
    "Amazon Fresh",
    "AmazonFresh BWI6",
    "AMZ*Seed-DS-01-Daily",
    "AMZ*Legion Athletics",
    "AMZ*CARFAX Vehicle H",
    "Amazon.com",
    "SQ*Local Terry AVE",
    "AMZN Mktp US*"
]

similar, unique = find_similar_merchants_simple(merchants, threshold=0.20)
print("## Simple Pairwise Comparison (Threshold 0.55) ☕️")
print(f"Similar Merchants:\n{similar}")
print(f"Unique Merchants:\n{unique}")

## Simple Pairwise Comparison (Threshold 0.55) ☕️
Similar Merchants:
[['AmazonFresh BWI6', 'SQ*Local Terry AVE', 'AMZ*CARFAX Vehicle H', 'Amazon.com', 'AMZ*Legion Athletics', 'Amazon Fresh', 'AMZN Mktp US*']]
Unique Merchants:
['AMZ*Seed-DS-01-Daily']


In [3]:
!pip install fuzzywuzzy python-levenshtein scikit-learn

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached python_levenshtein-0.27.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached levenshtein-0.27.3-cp311-cp311-win_amd64.whl.metadata (3.7 kB)
  Using cached rapidfuzz-3.14.3-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
Using cached python_levenshtein-0.27.3-py3-none-any.whl (9.5 kB)
Using cached levenshtein-0.27.3-cp311-cp311-win_amd64.whl (94 kB)
Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl (8.9 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached rapidfuzz-3.14.3-cp311-cp311-win_amd64.whl (1.5 MB)



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.cluster import DBSCAN
import numpy as np

def fuzzy_clustering_dbscan(merchant_list, eps=0.2, min_samples=2):
    """
    Groups similar merchant names using Levenshtein distance 
    and the DBSCAN clustering algorithm.
    
    Args:
        merchant_list (list): The list of merchant names (strings).
        eps (float): The maximum distance between two samples for one 
                     to be considered as in the neighborhood of the other (0.0 to 1.0).
        min_samples (int): The number of samples (or total weight) in a neighborhood 
                           for a point to be considered as a core point.
    
    Returns:
        tuple: (list of similar groups, list of unique names)
    """
    n = len(merchant_list)
    # 1. Create a Distance Matrix
    # We use a distance based on the normalized Levenshtein ratio (0.0 to 1.0)
    # Distance = 1 - (fuzz.ratio / 100)
    # The higher the fuzz.ratio, the lower the distance (closer to 0.0)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            # Calculate the distance (1 - similarity)
            # We use token_set_ratio as it is more robust to word order/extra words
            similarity = fuzz.token_set_ratio(merchant_list[i].lower(), merchant_list[j].lower()) / 100
            distance = 1 - similarity
            distance_matrix[i, j] = distance_matrix[j, i] = distance

    # 2. Apply DBSCAN Clustering
    # The metric='precomputed' tells DBSCAN to use the distance_matrix directly
    # `eps` is the max distance allowed for names to be in the same cluster.
    db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(distance_matrix)
    labels = db.labels_

    # 3. Group and Separate Results
    
    # Groups are identified by their label (0, 1, 2, ...), 
    # and "noise" (unique names) have a label of -1.
    df = pd.DataFrame({'name': merchant_list, 'cluster': labels})
    
    similar_merchants = []
    unique_merchants = []
    
    # Iterate through unique labels (clusters)
    for label in np.unique(labels):
        group = df[df['cluster'] == label]['name'].tolist()
        
        if label == -1 or len(group) == 1:
            # -1 is noise (unique), or a group with only one member
            unique_merchants.extend(group)
        else:
            # A valid cluster with multiple members
            similar_merchants.append(group)
            
    return similar_merchants, unique_merchants

# --- Example Usage ---
merchants = [
    "Starbucks Coffee",
    "StarBucks",
    "The Home Depot",
    "HomeDepot",
    "Best Buy Co.",
    "Best Buy",
    "Lowe's",
    "McDonald's",
    "McDonalds Corp" 
]

# A lower `eps` (e.g., 0.1) means they must be very similar (90% ratio or more)
# A higher `eps` (e.g., 0.3) means they can be less similar (70% ratio or more)
similar_dbscan, unique_dbscan = fuzzy_clustering_dbscan(merchants, eps=0.2, min_samples=2)

print("\n## Fuzzy Clustering with DBSCAN (EPS 0.2) 📊")
print(f"Similar Merchants:\n{similar_dbscan}")
print(f"Unique Merchants:\n{unique_dbscan}")


## Fuzzy Clustering with DBSCAN (EPS 0.2) 📊
Similar Merchants:
[['Starbucks Coffee', 'StarBucks'], ['Best Buy Co.', 'Best Buy']]
Unique Merchants:
['The Home Depot', 'HomeDepot', "Lowe's", "McDonald's", 'McDonalds Corp']


In [10]:
# --- Example Usage ---
merchants = [
    "Amazon Fresh",
    "AmazonFresh BWI6",
    "AMZ*Seed-DS-01-Daily",
    "AMZ*Legion Athletics",
    "AMZ*CARFAX Vehicle H",
    "Amazon.com",
    "SQ*Local Terry AVE",
    "AMZN Mktp US*"
]

# A lower `eps` (e.g., 0.1) means they must be very similar (90% ratio or more)
# A higher `eps` (e.g., 0.3) means they can be less similar (70% ratio or more)
similar_dbscan, unique_dbscan = fuzzy_clustering_dbscan(merchants, eps=0.5, min_samples=1)

print("\n## Fuzzy Clustering with DBSCAN (EPS 0.2) 📊")
print(f"Similar Merchants:\n{similar_dbscan}")
print(f"Unique Merchants:\n{unique_dbscan}")


## Fuzzy Clustering with DBSCAN (EPS 0.2) 📊
Similar Merchants:
[['Amazon Fresh', 'AmazonFresh BWI6', 'Amazon.com', 'AMZN Mktp US*'], ['AMZ*Legion Athletics', 'AMZ*CARFAX Vehicle H']]
Unique Merchants:
['AMZ*Seed-DS-01-Daily', 'SQ*Local Terry AVE']
